### AJUSTE HIPERPARAMETROS MODELO DE ARBOLES ALEATORIOS (RF)

### Cargar los datos ya limpiados y analizados en el Notebook analisis data y cargar librerias necesarias

In [4]:
!pip install -q -U keras-tuner

In [1]:

import numpy as np
import pandas as pd
from keras import backend as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
datos=pd.read_csv("Data_clean_Completo4.csv")



### Preprocesamiento de datos

In [2]:

variables_categoricas = ["Cation", "Anion","T_Comportamiento"]
variables_numericas = ["Solute_Mol_Mass", "Num_Cations", "Num_Anions", "Temperature_C", "Solvent_Mol_Mass", "Molal"]
variables_a_predecir = ["Molal_Activity_Coefficient", "Osmotic_Coefficient"]


column_transformer = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(), variables_categoricas),
        ("num", StandardScaler(), variables_numericas)
    ])




### División de datos en conjuntos de entrenamiento y prueba

In [3]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(datos, test_size=0.2, random_state=42, stratify=datos['Id_solucion'])
X_test = column_transformer.fit_transform(test_data[variables_categoricas + variables_numericas])
y_test = test_data[variables_a_predecir].values
X_train = column_transformer.fit_transform(train_data[variables_categoricas + variables_numericas])
y_train = train_data[variables_a_predecir].values
X_train=X_train.toarray()


### Optimizacion RFR

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# Definir hiperparámetros y espacio de búsqueda
param_dist = {
    'n_estimators': [50, 100, 150, 200, 250, 300],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5, 10],
}

# Crear modelo de RFR
model = RandomForestRegressor(random_state=500)

# Definir objeto de búsqueda aleatoria
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=10,  # número de configuraciones de hiperparámetros que se probarán
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=1,
)

# Ejecutar búsqueda aleatoria
random_search.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados
print("Best parameters found: ", random_search.best_params_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END max_depth=20, min_samples_split=10, n_estimators=300; total time=  11.0s
[CV] END max_depth=20, min_samples_split=10, n_estimators=300; total time=  10.5s
[CV] END max_depth=20, min_samples_split=10, n_estimators=300; total time=  10.1s
[CV] END max_depth=10, min_samples_split=10, n_estimators=100; total time=   2.4s
[CV] END max_depth=10, min_samples_split=10, n_estimators=100; total time=   2.2s
[CV] END max_depth=10, min_samples_split=10, n_estimators=100; total time=   3.3s
[CV] END max_depth=20, min_samples_split=5, n_estimators=150; total time=   5.1s
[CV] END max_depth=20, min_samples_split=5, n_estimators=150; total time=   5.0s
[CV] END max_depth=20, min_samples_split=5, n_estimators=150; total time=   5.9s
[CV] END max_depth=20, min_samples_split=10, n_estimators=50; total time=   1.9s
[CV] END max_depth=20, min_samples_split=10, n_estimators=50; total time=   1.8s
[CV] END max_depth=20, min_samples_split=1